convert csv to txt

In [ ]:
with open('test.csv', 'r') as inp, open('test.txt', 'w') as out:
    for line in inp:
        line = line.replace(',', ':')
        out.write(line)

In [ ]:
import string
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	#stop_words = set(stopwords.words('english'))
	#tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
# load the document
filename = 'neg.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

In [ ]:
# load all docs in a directory
def process_docs(directory, is_trian):
	documents = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load the doc
		doc = load_doc(path)
		# clean doc
		tokens = clean_doc(doc)
		# add to list
		documents.append(tokens)
	return documents

In [ ]:
import os

# later in code...
os.listdir(directory)
os.path.join("string1", "string2")
os.isfile(txt_sentoken.rar)

In [ ]:
!unzip txt_sentoken.zip

In [ ]:
import pandas as pd

In [ ]:
from os import listdir
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('gdrive/My Drive/txt_sentoken/neg',delimiter=';')
#negative_docs = drive.mount("gdrive/My Drive/txt_sentoken/neg", True)

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os 
negative_docs = os.path.join('/content/drive/My Drive/txt_sentoken/neg')
positive_docs = os.path.join('/content/drive/My Drive/txt_sentoken/pos')

In [ ]:
import os
from os import listdir
from google.colab import drive
#drive.mount('/content/drive')
os.chdir('gdrive/My Drive/txt_sentoken/neg')

In [ ]:
#negative_docs = os.path.join('/content/drive/My Drive/txt_sentoken/neg')

trainX = negative_docs + positive_docs
trainy = [0 for _ in range(9)] + [1 for _ in range(9)]
save_dataset([trainX,trainy], 'train.pkl')

In [ ]:
def save_dataset(dataset, filename):
	dump(dataset, open(filename, 'wb'))
	print('Saved: %s' % filename)

In [ ]:
from tensorflow.keras import optimizers
#from tensorflow.python.keras.optimizers import TFOptimizer
import numpy as np
from nltk.corpus import stopwords
import string
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
#from tensorflow.keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate

In [ ]:
from pickle import load
def load_dataset(filename):
	return load(open(filename, 'rb'))
 
trainLines, trainLabels = load_dataset('train.pkl')
testLines, testLabels = load_dataset('test.pkl')

def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

  # calculate the maximum document length
def max_length(trainLines):
  return max([len(s.split()) for s in trainLines])

In [ ]:
# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
testLines, testLabels = load_dataset('test.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
length = 50
#vocab_size=1000
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines,20)
print(trainX.shape)

In [ ]:
trainLabels=np.reshape(trainLabels,(1,40, 1))
trainX=np.reshape(trainX,(1,40, 50))

In [ ]:
#from tensorflow.keras.models import Sequential, save_model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=1)(drop1)
	flat1 = Flatten()(pool1)
 

	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=1)(drop2)
	flat2 = Flatten()(pool2)


 
	merged = concatenate([flat1, flat2])
	
    # interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)

	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

	# summarize
	print(model.summary())
	#plot_model(model, show_shapes=True, to_file='/Users/user/Desktop/txt_sentoken/multichannel.png')
    #model.save("my_h5_model.h5")
   
	return model



In [ ]:
# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = 40
0
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
trainX=np.reshape(trainX,(40, 100))
trainLabels=np.reshape(trainLabels,(40, 1))#
trainX = np.array_split(trainX, 2, axis=1)
#print(trainX.shape)
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX], array(trainLabels), epochs=10, batch_size=16)
# save the model

In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

batch_size = 128
num_classes = 10
epochs = 12

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), activation="relu", input_shape=(28, 28, 1) ))
model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2) ))

model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()


In [ ]:
import numpy as np
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

# load a clean dataset
def load_dataset(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the maximum document length
def max_length(lines):
  for s in lines:
    s=str(s)
    Type = s.split(",")
  return max([len(Type) for s in lines])
	#return max([len(Type) for s in lines])

# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 40)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 40)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size,40)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
#length = 40
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(length)
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
trainX=np.reshape(trainX,(5, 1108700))
trainLabels=np.reshape(trainLabels,(5, 8))
print(trainX.shape)

# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX], array(trainLabels), epochs=10, batch_size=16)
# save the model
model.save('model.h5')


55435
Max document length: 55435
Vocabulary size: 5411
(5, 1108700)
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 55435)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 55435)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 55435)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 55435, 40)    216440      input_1[0][0]                    
___________________

AssertionError: ignored

In [ ]:
trainX = np.array_split(trainX, 2, axis=1)
trainX

In [ ]:
# evaluate model on training dataset
loss, acc = model.evaluate([trainX,trainX], array(trainLabels), verbose=0)
print('Train Accuracy: %f' % (acc*100))
 
# evaluate model on test dataset dataset
loss, acc = model.evaluate([testX,testX],array(testLabels), verbose=0)
print('Test Accuracy: %f' % (acc*100))

AssertionError: ignored

In [ ]:
model = Sequential()
model.add(Convolution2D(filters = 32, kernel_size = (3, 3),
          input_shape = (64, 64, 3),
          activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Convolution2D(32, 3, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer='adam',
                        loss = 'binary_crossentropy',
                        metrics = ['accuracy'])

In [ ]:
import numpy as np
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

# load a clean dataset
def load_dataset(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the maximum document length
def max_length(lines):
  for s in lines:
    s=str(s)
    Type = s.split(",")
  return max([len(Type) for s in lines])
	#return max([len(Type) for s in lines])

# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 40)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 40)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size,40)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
#length = 40
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(length)
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
#trainX=np.reshape(trainX,(5, 1108700))
#trainLabels=np.reshape(trainLabels,(5, 8))
print(trainX.shape)

# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX], array(trainLabels), epochs=10, batch_size=16)
# save the model
model.save('model.h5')


UnpicklingError: ignored